In [1]:
#加载云盘

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


加载库

In [3]:
import numpy as np
import re

加载数据,初始化数据集

In [4]:
for i in range(1,2):
  token=open("/content/drive/MyDrive/机器学习/朴素贝叶斯/spam/%d.txt" %i).read().split(" ")   #读取spam邮件内容
  token=[tk for tk in token if len(tk)>2]
  print(token)
#split不能处理部分字符，试试re方法

['---', 'Codeine', '15mg', 'for', '$203.70', 'VISA', 'Only!!!', '--\n\n--', 'Codeine', '(Methylmorphine)', 'narcotic', '(opioid)', 'pain', 'reliever\n--', 'have', '15mg', '30mg', 'pills', '30/15mg', 'for', '$203.70', '60/15mg', 'for', '$385.80', '90/15mg', 'for', '$562.50', 'VISA', 'Only!!!', '---']


In [5]:
for i in range(1,2):
  token=open("/content/drive/MyDrive/机器学习/朴素贝叶斯/spam/%d.txt" %i).read()    #读取spam邮件内容
  find_words=re.compile("\\w*")
  token=[tk.lower() for tk in re.findall(find_words,token) if len(tk)>2]      #除了字母和数字，其他字符不要,转变为小写
token

['codeine',
 '15mg',
 'for',
 '203',
 'visa',
 'only',
 'codeine',
 'methylmorphine',
 'narcotic',
 'opioid',
 'pain',
 'reliever',
 'have',
 '15mg',
 '30mg',
 'pills',
 '15mg',
 'for',
 '203',
 '15mg',
 'for',
 '385',
 '15mg',
 'for',
 '562',
 'visa',
 'only']

In [6]:
#初始化数据集
doclist=[]    #每一封邮件的切分文本
classlist=[]   #每封邮件的类别，0是spam，1是ham
fulltext=[]   #一个总的词汇列表
for i in range(1,26):
  find_words=re.compile("\\w*")
  token0=open("/content/drive/MyDrive/机器学习/朴素贝叶斯/spam/%d.txt" %i,encoding="gbk").read()    #读取spam邮件内容
  token0=[tk.lower() for tk in re.findall(find_words,token0) if len(tk)>2]      #除了字母和数字，其他字符不要,转变为小写
  doclist.append(token0)
  classlist.append(0)                                   #class0
  fulltext.extend(token0)

  token1=open("/content/drive/MyDrive/机器学习/朴素贝叶斯/ham/%d.txt" %i,encoding="gbk").read()    #读取ham邮件内容
  token1=[tk.lower() for tk in re.findall(find_words,token1) if len(tk)>2]      
  doclist.append(token1)
  classlist.append(1)                                   #class1
  fulltext.extend(token1)

In [7]:
print(doclist)

[['codeine', '15mg', 'for', '203', 'visa', 'only', 'codeine', 'methylmorphine', 'narcotic', 'opioid', 'pain', 'reliever', 'have', '15mg', '30mg', 'pills', '15mg', 'for', '203', '15mg', 'for', '385', '15mg', 'for', '562', 'visa', 'only'], ['peter', 'with', 'jose', 'out', 'town', 'you', 'want', 'meet', 'once', 'while', 'keep', 'things', 'going', 'and', 'some', 'interesting', 'stuff', 'let', 'know', 'eugene'], ['hydrocodone', 'vicodin', 'brand', 'watson', 'vicodin', '750', '195', '120', '570', 'brand', 'watson', '750', '195', '120', '570', 'brand', 'watson', '325', '199', '120', '588', 'noprescription', 'required', 'free', 'express', 'fedex', 'days', 'delivery', 'for', 'over', '200', 'order', 'major', 'credit', 'cards', 'check'], ['yay', 'you', 'both', 'doing', 'fine', 'working', 'mba', 'design', 'strategy', 'cca', 'top', 'art', 'school', 'new', 'program', 'focusing', 'more', 'right', 'brained', 'creative', 'and', 'strategic', 'approach', 'management', 'the', 'way', 'done', 'today'], ['yo

In [8]:
print(classlist)      #两种类型交叉了，不知道处理起来方不方便，先不管><

[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]


In [9]:
#fulltext去除重复单词
print(len(fulltext))

fulltext=list(set(fulltext))
print(len(fulltext))

1762
694


根据词汇表，将每个文本向量化

In [10]:
doclist_vec=[]
for data in doclist:
  returnVec=[0]*len(fulltext)  #长度和全词汇列表一致的0向量
  for word in data:
    if word in fulltext:
      returnVec[fulltext.index(word)]=1
  doclist_vec.append(returnVec)
len(doclist_vec[0])

694

拆分训练集与测试集

In [11]:
import random

In [12]:
rate=0.12 #按照12%的比率拆分
train_data=[]
train_label=[]
test_data=[]
test_label=[]
for i in range(len(doclist_vec)):
  n=random.random()
  if n<=rate:
    test_data.append(doclist_vec[i])
    test_label.append(classlist[i])
  else:
    train_data.append(doclist_vec[i])
    train_label.append(classlist[i])

#如果数据集比较大，就地拆分比较好，不会浪费内存
#random拆分不会严格按照0.1的比例，但是当样本数量足够多时，测试集和训练集的比例会更接近0.1--->蒙特卡洛模拟

In [13]:
print(len(train_data))
print(len(train_label))
print(len(test_data))
print(len(test_label))    #样本量太少，可能会存在测试集为0的问题

46
46
4
4


直接使用sklearn的高斯贝叶斯算法进行训练

In [14]:
#直接用sklearn训练
from sklearn.naive_bayes import GaussianNB

clf=GaussianNB()
clf.fit(train_data,train_label)
clf.score(test_data,test_label)   #直接用sklearn的正确率接近1，可以多次迭代测试正确率

1.0

手写贝叶斯算法

In [57]:
def trainNB(train_data,train_label):
  sample_num=len(train_data)     #训练集样本数
  len_vec=len(train_data[0])     #每个样本的词条向量的长度，ps每个样本长度都一样
  p_class1=sum(train_label)/sample_num  #ham类型label为1，所以p_class1是非侮辱类邮件的概率
  p_class0=1-p_class1           #侮辱类邮件的概率
  lpls_up_c0=np.ones(len_vec)       #拉普拉斯平滑，防止零概率，将每个词条初始化为1，分母初始化为2
  lpls_up_c1=np.ones(len_vec)
  lpls_down_c0=2
  lpls_down_c1=2
  for i in range(sample_num):     #统计每个类别中，各词条出现的概率
    if train_label[i]==0:       #侮辱类样本
      lpls_up_c0+=train_data[i]
      lpls_down_c0+=1        #所有该分类样本的个数作为分母(直接加1)or所有词汇出现的总次数sum(train_data[i])？？？
    else:
      lpls_up_c1+=train_data[i]
      lpls_down_c1+=1
  vec_p0=np.log(lpls_up_c0/lpls_down_c0)
  vec_p1=np.log(lpls_up_c1/lpls_down_c1)
  return vec_p0,vec_p1,p_class0

In [58]:
#构建分类器
def clsNB(train_data,vec_p0,vec_p1,p_class0):
  returnvec=[]
  for vec in train_data:
    p1=sum(vec*vec_p1)+np.log(1-p_class0)        #对应元素相乘。logA * B = logA + logB，所以这里加上log(pClass1)
    p0=sum(vec*vec_p0)+np.log(p_class0)
    if p1>p0:
      returnvec.append(1)
    else:
      returnvec.append(0)
  return returnvec

In [59]:
vec_p0,vec_p1,p_class0=trainNB(train_data,train_label)
pre_label=clsNB(train_data,vec_p0,vec_p1,p_class0)
print(pre_label)
print(train_label)

[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]


In [62]:
pre_label=clsNB(test_data,vec_p0,vec_p1,p_class0)
print(pre_label)
print(test_label)

[1, 1, 1, 0]
[1, 1, 1, 0]
